In [ ]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import plotly.express as px
import random
import folium
from folium import plugins as folium_plugins
from shapely.geometry import MultiPoint

In [ ]:
# admin dasar
admin_gdf = gpd.read_file('../data/SHP/Administrasi.shp')
admin_gdf['centroid_latitude'] = admin_gdf['geometry'].centroid.y
admin_gdf['centroid_longitude'] = admin_gdf['geometry'].centroid.x
bintuni_dasar_gdf = admin_gdf.dissolve(by='Distrik').reset_index()
bintuni_dasar_gdf

# sagu
sagu_gdf = gpd.read_file('../data/SHP/Daerah_Sagu.shp')
# projected_crs = 'EPSG:3857' #Untuk Web Mercator (digunakan oleh OpenStreetMap, Google Maps, dan lainnya):
# sagu_gdf = sagu_gdf.to_crs(projected_crs)
sagu_gdf['centroid_latitude'] = sagu_gdf['geometry'].centroid.y
sagu_gdf['centroid_longitude'] = sagu_gdf['geometry'].centroid.x
sagu_gdf = sagu_gdf.dissolve(by='Distrik').reset_index()

In [ ]:
sagu_gdf

In [ ]:
# print(titik_tengah)
# POINT (133.3460630600616 -2.033975531897878) #benar
# POINT (310145.42612301995 9756361.519166619)

In [ ]:
m = folium.Map([-2.033975531897878, 133.346063060061], zoom_start=9)

# folium.Choropleth(
#     geo_data=admin_gdf,
#     fill_opacity=0.3,
#     line_weight=2,
# ).add_to(m)

# m
stripes = folium_plugins.pattern.StripePattern(angle=-45).add_to(m)

folium.GeoJson(
    bintuni_dasar_gdf,
    name='Area Teluk Bintuni',
    style_function=lambda feature: {
        "fillColor": "#e6f2df",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
        'lineJoin': 'round',
        # 'dashArray': '5, 5',
    },
    highlight_function=lambda feature: {
        "fillColor": "#d4decf",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['Distrik'],
        sticky=False,
    ),
    smooth_factor=2,
    ).add_to(m)

folium.GeoJson(
    sagu_gdf,
    name='Area Hutan Sagu',
    style_function=lambda feature: {
        "fillColor": "#009b00",
        "color": "#008200",
        # "fillPattern": stripes,
        "weight": 0.5,
        "fillOpacity": 0.7,
        'lineJoin': 'round',
    },
    highlight_function=lambda feature: {
        "fillColor": "#003e00",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['Penggunaan', 'Distrik', 'Luas'],
        sticky=False,
    ),
    smooth_factor=2,
    ).add_to(m)

folium_plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)
folium.TileLayer("esri-worldimagery").add_to(m)
folium.TileLayer("cartodbpositron").add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
m.save('../map-html/analisis_sagu.html')